In [1]:
import csv
from cv2 import imread,cvtColor
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
file = open('driver_imgs_list.csv')
lst = csv.reader(file)
file_list = []
for i in lst:
    file_list.append(i)
file_list.pop(0)
w = 160
h = 192
input_size = w * h

In [3]:
def read_file(file):
    img = cv2.imread('train/'+file[1]+'/'+file[2],0)
    img = cv2.resize(img,(w,h), interpolation=cv2.INTER_LINEAR)
    img = img.astype(np.float32)/255.0
    label = int(file[1][1])
    return img,label
def show(array):
    plt.figure()
    plt.imshow(array.reshape([h,w]), cmap="gray")
    plt.show()
def get_batch(batch_size):
    j = np.random.randint(0,20788,batch_size)
    batch = np.zeros((batch_size,h,w,1))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k,:,:,0],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label
def get_test_batch(batch_size):
    j = np.random.randint(20788,22424,batch_size)
    batch = np.zeros((batch_size,h,w,1))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k,:,:,0],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label

In [4]:
## 
X = tf.placeholder (dtype=tf.float32,shape=[None, h,w,1])
Y = tf.placeholder(dtype=tf.float32,shape=[None,10])
####
####
#con0= tf.layers.conv2d(inputs=X,kernel_size=(3,3),filters=16,activation=tf.nn.relu,strides=1,padding='same')
#con1 = tf.layers.conv2d(inputs=con0,kernel_size=(3,3),filters=16,activation=tf.nn.relu,strides=1,padding='same')
#max1 = tf.layers.max_pooling2d(inputs=con1,pool_size=2,strides=2)
####
con0 = tf.layers.conv2d(inputs=X,kernel_size=(11,11),filters=16,activation=tf.nn.relu,strides=1,padding='same')
max0 = tf.layers.max_pooling2d(inputs=con0,pool_size=2,strides=2)
########
con1 = tf.layers.conv2d(inputs=max0,kernel_size=(7,7),filters=16,activation=tf.nn.relu,strides=1,padding='same')
max1 = tf.layers.max_pooling2d(inputs=con1,pool_size=2,strides=2)
#####
con2 = tf.layers.conv2d(inputs=max1,kernel_size=(5,5),filters=16,activation=tf.nn.relu,strides=1,padding='same')
max2 = tf.layers.max_pooling2d(inputs=con2,pool_size=2,strides=2)
#####
con3 = tf.layers.conv2d(inputs=max2,kernel_size=(3,3),filters=16,activation=tf.nn.relu,strides=1,padding='same')
max3 = tf.layers.max_pooling2d(inputs=con3,pool_size=2,strides=2)
#####
con4 = tf.layers.conv2d(inputs=max3,kernel_size=(3,3),filters=16,activation=tf.nn.relu,strides=1,padding='same')
max4 = tf.layers.max_pooling2d(inputs=con4,pool_size=2,strides=2)
#####
#######################################
fc_1 = tf.reshape(max4,[-1,480])
dr = tf.placeholder(dtype=tf.float32,shape=[])
fc_1_d = tf.nn.dropout(fc_1,keep_prob=dr)
#######################################
fc_2 = tf.layers.dense(inputs=fc_1_d,units=2000,activation=tf.nn.relu)
fc_2_d = tf.nn.dropout(fc_2,keep_prob=dr)
#######################################
fc_3 = tf.layers.dense(inputs=fc_2_d,units=10)
#######################################
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=fc_3))
optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)
correctPred = tf.equal(tf.argmax(fc_3,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [5]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [9]:
for i in range(3000):
    batch_x,batch_y = get_batch(16)
    _,l,acc = sess.run([optimizer,loss,accuracy],feed_dict={X: batch_x,Y:batch_y,dr:0.5})
    if i % 100 == 0:
        print(l,acc)
        total_acc = 0
        for i in range(100):
            batch_x,batch_y = get_test_batch(16)
            l,acc = sess.run([loss,accuracy],feed_dict={X: batch_x,Y:batch_y,dr:1})
            total_acc += acc
        print(total_acc/100)

0.74514294 0.75
0.406875
0.41131315 0.75
0.4125
0.8606043 0.6875
0.4475
0.46515852 0.8125
0.401875
0.5484531 0.75
0.3925
0.74581623 0.8125
0.436875
0.6734744 0.75
0.424375
0.98448557 0.5625
0.435
0.37204355 0.875
0.433125
0.39217874 0.8125
0.401875
0.34944546 0.9375
0.400625
0.3630026 0.9375
0.399375
0.5022036 0.8125
0.405625
0.5515971 0.875
0.438125
0.41837394 0.875
0.4125
0.6541947 0.8125
0.415
0.10506964 1.0
0.43875
0.73756874 0.75
0.443125
0.75459814 0.6875
0.39875
0.5317628 0.875
0.3825
0.25554767 0.875
0.399375
0.33756194 0.875
0.4075
0.5755125 0.8125
0.360625
0.5009926 0.875
0.42375
0.535323 0.75
0.3475
0.25700542 0.875
0.361875
0.14400485 0.9375
0.383125
0.34697258 0.8125
0.42625
0.58792025 0.875
0.400625
0.44366732 0.875
0.379375


In [7]:
total_acc = 0
for i in range(100):
    batch_x,batch_y = get_test_batch(16)
    l,acc = sess.run([loss,accuracy],feed_dict={X: batch_x,Y:batch_y,dr:1})
    total_acc += acc
print(total_acc/100)

0.4075


In [8]:
sess.run(max5,feed_dict={X: batch_x,Y:batch_y,dr:1}).shape


NameError: name 'max5' is not defined